# The Battle of Neighborhoods in Dhaka City
### A Data Science project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

The business problem that is addressed in this notebook is that, if a person wants to open a new **coffee shop** in Dhaka city in **Bangladesh**, then what are the things that he/she has to look into before opening the shop. Here, by analyzing and exploring all of the thanas in **Dhaka** city, he can get useful insights about the venues present in the thanas. If he/she can find a thana where the availability of a coffee shop is rare currently, he/she could try to establish one in that neighborhood/thana. Also, he/she has to explore the neighboring thanas to get better insight for his/her business. In this case, the **stakeholders** are **himself/herself** and the people in the thanas. As he/she will be the **owner** of the coffee shop, and he/she wants to make profit off of it, he/she needs to analyze all the thanas near the city. So, he/she will be the **internal stakeholder**. And **the customer** will be the **consumers**. The popularity and prosperity of his/her business will very much depend of the customers' mood, whether they like the coffee shop or not, whether they like the services given by the employees or not. So, the customers will be the **external stakeholder** of the business.

## Data <a name="data"></a>

The dataset that I am working on is the Dhaka City **thanas** and **suboffices** according to their **postal codes**. It has been downloaded from the wikipedia page: [Dhaka Postal codes](https://en.wikipedia.org/wiki/List_of_postal_codes_in_Bangladesh). To scrape the webpage, I have used the **"beautifulsoup4"** library. The dataset consists of **four columns**, namely, **District** ==> refers to the District in which Dhaka city is in, **Thana** ==> the Thanas present in the city, and **Suboffice** ==> the Suboffices present under the Thanas and **Postal Codes** ==> according to the suboffices.
To explore each of the thanas, where all of the coffee shops, parks, restaurants and other venues, the **Foursquare API** has been used. To use the Foursquare API I needed the latitude and the longitude values of each of the thanas. The **latitude** and the **longitude** values are collected using **geocoder, geopy library**. 

### Part 00: Importing Libraries

In [1]:
#importing important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1f             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
!pip install beautifulsoup4

In [3]:
!pip install lxml

In [4]:
!pip install requests

In [5]:
from bs4 import BeautifulSoup
import requests

### Part 01: Generating the data

In [6]:
#Getting the source data from wikipedia page
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_in_Bangladesh').text

In [7]:
#Using BeautifulSoup4 to read the data
soup = BeautifulSoup(source, 'lxml')

In [8]:
#print(soup.prettify())

In [9]:
#Capturing the data table
table = soup.find("table", attrs={"class":"wikitable"})

In [10]:
table

<table class="wikitable">
<tbody><tr>
<th style="text-align: center; font-weight: bold;">District
</th>
<th style="text-align: center; font-weight: bold;">Thana
</th>
<th style="text-align: center; font-weight: bold;">SubOffice
</th>
<th style="text-align: center; font-weight: bold;">Post Code<sup class="reference" id="cite_ref-dhaka_1-0"><a href="#cite_note-dhaka-1">[1]</a></sup>
</th></tr>
<tr>
<td>Dhaka
</td>
<td>Dhaka
</td>
<td>Dhaka Cantonment--TSO
</td>
<td style="text-align: center;">1206
</td></tr>
<tr>
<td>Dhaka
</td>
<td>Dhamrai
</td>
<td>Dhamrai
</td>
<td style="text-align: center;">1350
</td>
<td>Dhaka
</td>
<td>Dhamrai
</td>
<td>Kalampur
</td>
<td style="text-align: center;">1351
</td></tr>
<tr>
<td>Dhaka
</td>
<td>Dhanmondi
</td>
<td>Jigatala TSO
</td>
<td style="text-align: center;">1209
</td></tr>
<tr>
<td>Dhaka
</td>
<td>Gulshan
</td>
<td>Banani TSO
</td>
<td style="text-align: center;">1213
</td></tr>
<tr>
<td>Dhaka
</td>
<td>Gulshan
</td>
<td>Badda
</td>
<td style="t

In [11]:
District = []
for i in table.find_all('tr'):
    for j in i.find_all('td'):
        District.append(j.text)
    District.append("***")  

In [12]:
new_dist = []
for i in District:
    new_dist.append(i.split('\n'))

In [13]:
for i in new_dist:
    if i==['***']:
        new_dist.remove(i)

In [14]:
new_dist

[['Dhaka', ''],
 ['Dhaka', ''],
 ['Dhaka Cantonment--TSO', ''],
 ['1206', ''],
 ['Dhaka', ''],
 ['Dhamrai', ''],
 ['Dhamrai', ''],
 ['1350', ''],
 ['Dhaka', ''],
 ['Dhamrai', ''],
 ['Kalampur', ''],
 ['1351', ''],
 ['Dhaka', ''],
 ['Dhanmondi', ''],
 ['Jigatala TSO', ''],
 ['1209', ''],
 ['Dhaka', ''],
 ['Gulshan', ''],
 ['Banani TSO', ''],
 ['1213', ''],
 ['Dhaka', ''],
 ['Gulshan', ''],
 ['Badda', ''],
 ['1212', ''],
 ['Dhaka', ''],
 ['Gulshan', ''],
 ['Gulshan Model Town', ''],
 ['1212', ''],
 ['Dhaka', ''],
 ['Jatrabari', ''],
 ['Dhania TSO', ''],
 ['1236', ''],
 ['Dhaka', ''],
 ['Joypara', ''],
 ['Joypara', ''],
 ['1331', ''],
 ['Dhaka', ''],
 ['Joypara', ''],
 ['Narisha', ''],
 ['1332', ''],
 ['Dhaka', ''],
 ['Joypara', ''],
 ['Palamganj', ''],
 ['1331', ''],
 ['Dhaka', ''],
 ['Keraniganj', ''],
 ['Ati', ''],
 ['1312', ''],
 ['Dhaka', ''],
 ['Keraniganj', ''],
 ['Dhaka Jute Mills', ''],
 ['1311', ''],
 ['Dhaka', ''],
 ['Keraniganj', ''],
 ['Kalatia', ''],
 ['1313', ''],
 ['Dhaka'

In [15]:
value = []
for i in new_dist:
    for j in i:
        if j=="":
            continue
        value.append(j)

In [16]:
District = []
Thana = []
Suboffice = []
PostCode = []
for i in range(0,1196,4):
    District.append(value[i])
for i in range(1,1197,4):
    Thana.append(value[i])
for i in range(2,1197,4):
    Suboffice.append(value[i])
for i in range(3,1197,4):
    PostCode.append(value[i])

In [17]:
List = list(zip(District, Thana, Suboffice, PostCode))

In [18]:
column_names = ['District', 'Thana', 'Suboffice', 'PostCode']
df = pd.DataFrame(List, columns=column_names)

In [19]:
df.head()

,District,Thana,Suboffice,PostCode
0,Dhaka,Dhaka,Dhaka Cantonment--TSO,1206
1,Dhaka,Dhamrai,Dhamrai,1350
2,Dhaka,Dhamrai,Kalampur,1351
3,Dhaka,Dhanmondi,Jigatala TSO,1209
4,Dhaka,Gulshan,Banani TSO,1213


In [20]:
df.tail()

,District,Thana,Suboffice,PostCode
294,Tangail,Sakhipur,Sakhipur,1950
295,Tangail,Tangail Sadar,Kagmari,1901
296,Tangail,Tangail Sadar,Korotia,1903
297,Tangail,Tangail Sadar,Purabari,1904
298,Tangail,Tangail Sadar,Santosh,1902


In [21]:
!pip install geocoder
from geopy.geocoders import Nominatim 
import geocoder # import geocoder

     |████████████████████████████████| 102kB 15.2MB/s ta 0:00:01


### Part 02: Adding Latitude and Longitude of the Neighborhoods the dataframe

In [22]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
city ="Dhaka"
country ="Bangladesh"
lat = []
long = []
for thana in df['Thana']:
    try:
        loc = geolocator.geocode(thana+','+city+','+ country)
        lat.append(loc.latitude)
        long.append(loc.longitude)
    except:
        lat.append(None)
        long.append(None)
# print("****")
# print(lat)
# print("")
# print(long)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [23]:
df['Latitude'] = lat
df['Longitude'] = long

In [24]:
df.dropna(inplace=True)

In [25]:
df_not_dhaka = df[df['District']!='Dhaka']
df.drop(df_not_dhaka.index, inplace=True)
df

,District,Thana,Suboffice,PostCode,Latitude,Longitude
0,Dhaka,Dhaka,Dhaka Cantonment--TSO,1206,23.759357,90.378814
1,Dhaka,Dhamrai,Dhamrai,1350,23.920162,90.210870
2,Dhaka,Dhamrai,Kalampur,1351,23.920162,90.210870
3,Dhaka,Dhanmondi,Jigatala TSO,1209,23.759357,90.378814
4,Dhaka,Gulshan,Banani TSO,1213,23.789987,90.411627
5,Dhaka,Gulshan,Badda,1212,23.789987,90.411627
6,Dhaka,Gulshan,Gulshan Model Town,1212,23.789987,90.411627
7,Dhaka,Jatrabari,Dhania TSO,1236,23.710423,90.434467
8,Dhaka,Joypara,Joypara,1331,23.607599,90.124962
9,Dhaka,Joypara,Narisha,1332,23.607599,90.124962


### Part 03: Using Geopy and Folium library to generate and explore the areas (Thanas) of Dhaka

In [26]:
#Using Geopy library to get Latitude and Longitude of Dhaka
address = 'Dhaka'

geolocator = Nominatim(user_agent="dhk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dhaka are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Dhaka are 23.7593572, 90.3788136.


In [27]:
# Creating a map of Neighborhoods (Thanas) using latitude and longitude values in Dhaka
map_dhaka = folium.Map(location=[latitude, longitude], zoom_start=10)

# Adding markers to the map
for lat, lng, thana, suboffice in zip(df['Latitude'], df['Longitude'], df['Thana'], df['Suboffice']):
    label = '{}, {}'.format(suboffice, thana)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dhaka)  
    
map_dhaka

In [28]:
df = df.groupby(['District','Thana','Latitude','Longitude'])['Suboffice'].apply(', '.join).reset_index()
df

,District,Thana,Latitude,Longitude,Suboffice
0,Dhaka,Dhaka,23.759357,90.378814,Dhaka Cantonment--TSO
1,Dhaka,Dhamrai,23.920162,90.210870,"Dhamrai, Kalampur"
2,Dhaka,Dhanmondi,23.759357,90.378814,Jigatala TSO
3,Dhaka,Gulshan,23.789987,90.411627,"Banani TSO, Badda, Gulshan Model Town"
4,Dhaka,Jatrabari,23.710423,90.434467,Dhania TSO
5,Dhaka,Joypara,23.607599,90.124962,"Joypara, Narisha, Palamganj"
6,Dhaka,Keraniganj,23.698189,90.350526,"Ati, Dhaka Jute Mills, Kalatia, Keraniganj"
7,Dhaka,Khilgaon,23.749702,90.417566,KhilgaonTSO
8,Dhaka,Khilkhet,23.830698,90.423599,KhilkhetTSO
9,Dhaka,Lalbag,23.718856,90.388780,Posta TSO


## Methodology <a name="methodology"></a>

<!-- As the business problem revolves around opening a coffee shop in a neighborhood in city of Vaughan in Canada, at first step the relevant **boroughs** are selected. The boroughs are: **North York, East York and York**. 

In the second step, **all the neighborhoods** that resides in the boroughs selected have been figured out. After that, using the **foursquare API**, the **venues** that are residing in those neighborhoods are found out.

In the next step, **filtering** of the neighborhoods have been done based on the criteria on the absence of coffee shops. This results in the neighborhoods in those boroughs that does not have any coffee shops in them.

Finally, a **clustering technique (k-means clustering)**  was used to find the clusters of similar neighborhoods. The clustering gives the necessary insight that is needed to find a place where if the coffee shop is established would result in **higher profit and customer satisfaction** for the owner.  -->

### Part 04: Using Foursquare API

In [29]:
#Defining Foursquare API Client ID, secret key, and version
CLIENT_ID = '5Q22GH3WURNDT2U33WNXOEGPESYBSSLTODWMIXUEHGYRXLXQ' 
CLIENT_SECRET = 'KVZBSGRAJELOR02BXJMSSZGPE2MLVPXQHNIH1VOJI0LDPOSM' 
VERSION = '20180605' 

print('My credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentials:
CLIENT_ID: 5Q22GH3WURNDT2U33WNXOEGPESYBSSLTODWMIXUEHGYRXLXQ
CLIENT_SECRET:KVZBSGRAJELOR02BXJMSSZGPE2MLVPXQHNIH1VOJI0LDPOSM


In [30]:
import json
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

In [31]:
# Get data of first neighborhood and use Foursquare API to get some insight of the venues of the neighborhood
thana_latitude = df['Latitude'][0] 
thana_longitude = df['Longitude'][0] 

thana_name = df['Thana'][0] 

print('Latitude and longitude values of {} are {}, {}.'.format(thana_name, 
                                                               thana_latitude, 
                                                               thana_longitude))

Latitude and longitude values of Dhaka are 23.7593572, 90.3788136.


In [32]:
# Setup API URL to explore venues near by Parkwoods
LIMIT = 150
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, thana_latitude, thana_longitude, VERSION, radius, LIMIT)
neighborhood_json = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = neighborhood_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Aarong,Arts & Crafts Store,23.758283,90.374102
1,Seventh Heaven,Café,23.758298,90.374111
2,Khamar Bari Mor,Plaza,23.759292,90.383501
3,Labanga,Restaurant,23.757643,90.374806
4,Fuwang Shwarma,Food,23.758093,90.374557


#### Getting nearby venues of the neighborhoods using Foursquare API

In [33]:
#Function to get nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 150;
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
          
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Thana', 
                  'Thana Latitude', 
                  'Thana Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
#Generate venues of Vaughan and printing the neighborhoods
print("Thanas in Dhaka:")
dhaka_venues = getNearbyVenues(names=df['Thana'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Thanas in Dhaka:
Dhaka
Dhamrai
Dhanmondi
Gulshan
Jatrabari
Joypara
Keraniganj
Khilgaon
Khilkhet
Lalbag
Mirpur
Mohammadpur
Motijheel
Nawabganj
Ramna
Sabujbag
Savar
Sutrapur
Tejgaon
Tejgaon Industrial Area
Uttara


In [35]:
dhaka_venues.head(10)

,Thana,Thana Latitude,Thana Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dhaka,23.759357,90.378814,Aarong,23.758283,90.374102,Arts & Crafts Store
1,Dhaka,23.759357,90.378814,Seventh Heaven,23.758298,90.374111,Café
2,Dhaka,23.759357,90.378814,Khamar Bari Mor,23.759292,90.383501,Plaza
3,Dhaka,23.759357,90.378814,Labanga,23.757643,90.374806,Restaurant
4,Dhaka,23.759357,90.378814,Fuwang Shwarma,23.758093,90.374557,Food
5,Dhaka,23.759357,90.378814,Shangshad Bhaban,23.757367,90.374543,Scenic Lookout
6,Dhamrai,23.920162,90.210870,Dhamrai Bazar,23.919938,90.211445,Market
7,Dhanmondi,23.759357,90.378814,Aarong,23.758283,90.374102,Arts & Crafts Store
8,Dhanmondi,23.759357,90.378814,Seventh Heaven,23.758298,90.374111,Café
9,Dhanmondi,23.759357,90.378814,Khamar Bari Mor,23.759292,90.383501,Plaza


### Part 05: Exploring all of the neighborhoods of Dhaka City

#### Finding out how many unique categories can be curated from all the returned venues

In [36]:
print('There are {} uniques categories.'.format(len(dhaka_venues['Venue Category'].unique())))

There are 48 uniques categories.


#### Checking the size of the resulting dataframe

In [37]:
dhaka_venues.shape

(104, 7)

#### Checking how many venues were returned for each neighborhood

In [38]:
dhaka_venues.groupby('Thana').count()

,Thana Latitude,Thana Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Thana,,,,,,
Dhaka,6,6,6,6,6,6
Dhamrai,1,1,1,1,1,1
Dhanmondi,6,6,6,6,6,6
Gulshan,40,40,40,40,40,40
Jatrabari,3,3,3,3,3,3
Khilgaon,6,6,6,6,6,6
Khilkhet,4,4,4,4,4,4
Lalbag,3,3,3,3,3,3
Mirpur,1,1,1,1,1,1


In [39]:
#checking out the list of different venue catagories
venue_category = list(dhaka_venues['Venue Category'].unique())
difrnt_venue_categories = pd.DataFrame(venue_category, columns=['Venue Category'])
difrnt_venue_categories

,Venue Category
0,Arts & Crafts Store
1,Café
2,Plaza
3,Restaurant
4,Food
5,Scenic Lookout
6,Market
7,Clothing Store
8,Hotel
9,Coffee Shop


#### Insights: 
We can see that their are 49 different category of venues present in different thanas of Dhaka city. Now lets find out how many thanas does not have any japanese restaurant in them.

<!-- #### Finding out the thanas that does not have a chinese restaurant -->

In [67]:
#Checking the thanas that doesnot have a chinese restaurant
# no_chn_thana_df = dhaka_venues[dhaka_venues['Venue Category']!='Chinese Restaurant'].reset_index(drop=True)
# no_chn_thana = list(temp.Thana.unique())
# no_chn_thana_df

In [68]:
#Thanas that don't have a chinese restaurant
# no_chn_thana

## Analysis <a name="analysis"></a>

### Part 06: Analysis on the thanas by each venue category

In [69]:
# one hot encoding
dhaka_venues_onehot = pd.get_dummies(dhaka_venues[['Venue Category']], prefix="", prefix_sep="")

# add thana column back to dataframe
dhaka_venues_onehot['Thana'] = dhaka_venues['Thana'] 

# move thana column to the first column
fixed_columns = [dhaka_venues_onehot.columns[-1]] + list(dhaka_venues_onehot.columns[:-1])
dhaka_venues_onehot = dhaka_venues_onehot[fixed_columns]

dhaka_venues_onehot.head()

,Thana,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Bakery,Bar,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,...,Pizza Place,Plaza,Portuguese Restaurant,Restaurant,Sandwich Place,Scenic Lookout,Shopping Mall,Social Club,Steakhouse,Sushi Restaurant
0,Dhaka,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dhaka,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Dhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Dhaka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [70]:
dhaka_venues_onehot.shape

(104, 49)

#### Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [71]:
dhaka_venues_grouped = dhaka_venues_onehot.groupby('Thana').mean().reset_index()

In [72]:
#checking the head of the dataframe
dhaka_venues_grouped.head(10)

,Thana,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Bakery,Bar,Breakfast Spot,Burger Joint,Bus Station,Bus Stop,...,Pizza Place,Plaza,Portuguese Restaurant,Restaurant,Sandwich Place,Scenic Lookout,Shopping Mall,Social Club,Steakhouse,Sushi Restaurant
0,Dhaka,0.166667,0.000,0.00,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.000000,0.166667,0.000,0.166667,0.000,0.166667,0.00,0.000,0.000,0.000
1,Dhamrai,0.000000,0.000,0.00,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000,0.000,0.000
2,Dhanmondi,0.166667,0.000,0.00,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.000000,0.166667,0.000,0.166667,0.000,0.166667,0.00,0.000,0.000,0.000
3,Gulshan,0.000000,0.025,0.05,0.0,0.00,0.025,0.025,0.000000,0.0,...,0.000000,0.000000,0.025,0.075000,0.025,0.000000,0.05,0.025,0.025,0.025
4,Jatrabari,0.000000,0.000,0.00,0.0,0.00,0.000,0.000,0.333333,0.0,...,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000,0.000,0.000
5,Khilgaon,0.000000,0.000,0.00,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000,0.000,0.000
6,Khilkhet,0.000000,0.000,0.00,0.0,0.25,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.25,0.000,0.000,0.000
7,Lalbag,0.000000,0.000,0.00,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.333333,0.000000,0.000,0.333333,0.000,0.000000,0.00,0.000,0.000,0.000
8,Mirpur,0.000000,0.000,0.00,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000,0.000,0.000
9,Mohammadpur,0.000000,0.000,0.20,0.0,0.00,0.000,0.000,0.000000,0.0,...,0.000000,0.000000,0.000,0.000000,0.000,0.000000,0.00,0.000,0.000,0.000


In [73]:
#Shape of the dataframe
dhaka_venues_grouped.shape

(17, 49)

#### Finding each neighborhood along with the top 5 most common venues

In [74]:
num_top_venues = 5

for hood in no_chn_thana_df_grouped['Thana']:
    print("----"+hood+"----")
    temp = dhaka_venues_grouped[dhaka_venues_grouped['Thana'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Dhaka----
                 venue  freq
0  Arts & Crafts Store  0.17
1       Scenic Lookout  0.17
2                 Food  0.17
3           Restaurant  0.17
4                 Café  0.17


----Dhamrai----
                  venue  freq
0                Market   1.0
1   Arts & Crafts Store   0.0
2  Arts & Entertainment   0.0
3           IT Services   0.0
4        Ice Cream Shop   0.0


----Dhanmondi----
                 venue  freq
0  Arts & Crafts Store  0.17
1       Scenic Lookout  0.17
2                 Food  0.17
3           Restaurant  0.17
4                 Café  0.17


----Gulshan----
               venue  freq
0         Restaurant  0.08
1     Clothing Store  0.08
2              Hotel  0.08
3  Indian Restaurant  0.08
4     Ice Cream Shop  0.05


----Jatrabari----
                 venue  freq
0         Intersection  0.33
1          Bus Station  0.33
2    Electronics Store  0.33
3  Arts & Crafts Store  0.00
4   Mexican Restaurant  0.00


----Khilgaon----
                       venu

#### Sorting the venues in descending order

In [75]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [90]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# creating columns according to number of top venues
columns = ['Thana']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# creating a new dataframe
dhaka_venues_sorted = pd.DataFrame(columns=columns)
dhaka_venues_sorted['Thana'] = dhaka_venues_grouped['Thana']

for ind in np.arange(no_chn_thana_df_grouped.shape[0]):
    dhaka_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dhaka_venues_grouped.iloc[ind, :], num_top_venues)

dhaka_venues_sorted.head()

,Thana,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dhaka,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
1,Dhamrai,Market,History Museum,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop,Department Store,College Bookstore
2,Dhanmondi,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
3,Gulshan,Clothing Store,Restaurant,Hotel,Indian Restaurant,Asian Restaurant,Shopping Mall,Ice Cream Shop,Korean Restaurant,Breakfast Spot,Burger Joint
4,Jatrabari,Electronics Store,Intersection,Bus Station,Sushi Restaurant,Coffee Shop,Food,Fast Food Restaurant,Donut Shop,Diner,Dessert Shop


### Part 07: Clustering the Neighborhoods

#### Running K-means clustering algorithm to cluster the neighborhoods

In [91]:
# setting number of clusters
kclusters = 5

dhaka_venues_grouped_clustering = dhaka_venues_grouped.drop('Thana', 1)

# running k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dhaka_venues_grouped_clustering)

# checking cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 2, 1], dtype=int32)

In [92]:
# adding clustering labels
dhaka_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dhaka_venues_merged = dhaka_venues

# merging dhaka_venues_grouped with dhaka_venues to add latitude/longitude for each thana
dhaka_venues_merged = dhaka_venues_merged.join(dhaka_venues_sorted.set_index('Thana'), on='Thana')

dhaka_venues_merged.head()

,Thana,Thana Latitude,Thana Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Dhaka,23.759357,90.378814,Aarong,23.758283,90.374102,Arts & Crafts Store,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
1,Dhaka,23.759357,90.378814,Seventh Heaven,23.758298,90.374111,Café,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
2,Dhaka,23.759357,90.378814,Khamar Bari Mor,23.759292,90.383501,Plaza,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
3,Dhaka,23.759357,90.378814,Labanga,23.757643,90.374806,Restaurant,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
4,Dhaka,23.759357,90.378814,Fuwang Shwarma,23.758093,90.374557,Food,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop


#### Creating a map for the clusters

In [93]:
# creating map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dhaka_venues_merged['Thana Latitude'], dhaka_venues_merged['Thana Longitude'], dhaka_venues_merged['Thana'], dhaka_venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Part 08: Examining the clusters

##### Cluster 0

In [94]:
dhaka_venues_merged.loc[dhaka_venues_merged['Cluster Labels'] == 0, dhaka_venues_merged.columns[[1] + list(range(5, dhaka_venues_merged.shape[1]))]]

,Thana Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,23.920162,90.211445,Market,0,Market,History Museum,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop,Department Store,College Bookstore


##### Cluster 1

In [95]:
dhaka_venues_merged.loc[dhaka_venues_merged['Cluster Labels'] == 1, dhaka_venues_merged.columns[[1] + list(range(5, dhaka_venues_merged.shape[1]))]]

,Thana Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,23.759357,90.374102,Arts & Crafts Store,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
1,23.759357,90.374111,Café,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
2,23.759357,90.383501,Plaza,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
3,23.759357,90.374806,Restaurant,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
4,23.759357,90.374557,Food,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
5,23.759357,90.374543,Scenic Lookout,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
7,23.759357,90.374102,Arts & Crafts Store,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
8,23.759357,90.374111,Café,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
9,23.759357,90.383501,Plaza,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop
10,23.759357,90.374806,Restaurant,1,Arts & Crafts Store,Restaurant,Café,Plaza,Food,Scenic Lookout,Bakery,Fast Food Restaurant,Electronics Store,Donut Shop


##### Cluster 2

In [96]:
dhaka_venues_merged.loc[dhaka_venues_merged['Cluster Labels'] == 2, dhaka_venues_merged.columns[[1] + list(range(5, dhaka_venues_merged.shape[1]))]]

,Thana Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,23.812229,90.361576,Department Store,2,Department Store,Sushi Restaurant,Steakhouse,Gym,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop


##### Cluster 3

In [97]:
dhaka_venues_merged.loc[dhaka_venues_merged['Cluster Labels'] == 3, dhaka_venues_merged.columns[[1] + list(range(5, dhaka_venues_merged.shape[1]))]]

,Thana Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,23.726597,90.421565,Restaurant,3,Restaurant,Burger Joint,Sushi Restaurant,Clothing Store,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop
76,23.726597,90.421597,Restaurant,3,Restaurant,Burger Joint,Sushi Restaurant,Clothing Store,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop
77,23.726597,90.422061,Burger Joint,3,Restaurant,Burger Joint,Sushi Restaurant,Clothing Store,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop
78,23.726597,90.422361,Restaurant,3,Restaurant,Burger Joint,Sushi Restaurant,Clothing Store,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop
79,23.726597,90.421555,Restaurant,3,Restaurant,Burger Joint,Sushi Restaurant,Clothing Store,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop


##### Cluster 4

In [98]:
dhaka_venues_merged.loc[dhaka_venues_merged['Cluster Labels'] == 4, dhaka_venues_merged.columns[[1] + list(range(5, dhaka_venues_merged.shape[1]))]]

,Thana Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
80,23.732714,90.404297,History Museum,4,History Museum,College Bookstore,Steakhouse,Gym,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop
81,23.732714,90.397304,College Bookstore,4,History Museum,College Bookstore,Steakhouse,Gym,Food,Fast Food Restaurant,Electronics Store,Donut Shop,Diner,Dessert Shop


### Part 09: Exploratory Analysis and visualization

#### Finding the venues in each cluster

In [107]:
clusterd_venues = dhaka_venues_merged.set_index('Cluster Labels')
clusterd_venues.iloc[:,0:6]

,Thana,Thana Latitude,Thana Longitude,Venue,Venue Latitude,Venue Longitude
Cluster Labels,,,,,,
1,Dhaka,23.759357,90.378814,Aarong,23.758283,90.374102
1,Dhaka,23.759357,90.378814,Seventh Heaven,23.758298,90.374111
1,Dhaka,23.759357,90.378814,Khamar Bari Mor,23.759292,90.383501
1,Dhaka,23.759357,90.378814,Labanga,23.757643,90.374806
1,Dhaka,23.759357,90.378814,Fuwang Shwarma,23.758093,90.374557
1,Dhaka,23.759357,90.378814,Shangshad Bhaban,23.757367,90.374543
0,Dhamrai,23.920162,90.210870,Dhamrai Bazar,23.919938,90.211445
1,Dhanmondi,23.759357,90.378814,Aarong,23.758283,90.374102
1,Dhanmondi,23.759357,90.378814,Seventh Heaven,23.758298,90.374111


#### Finding top three venues in each cluster

In [109]:
for cluster_label, cluster_df in dhaka_venues_merged.groupby("Cluster Labels"):
    print(cluster_label)
    print(cluster_df["Venue Category"].value_counts().head(3))

0
Market    1
Name: Venue Category, dtype: int64
1
Restaurant          8
Shopping Mall       6
Department Store    5
Name: Venue Category, dtype: int64
2
Department Store    1
Name: Venue Category, dtype: int64
3
Restaurant      4
Burger Joint    1
Name: Venue Category, dtype: int64
4
College Bookstore    1
History Museum       1
Name: Venue Category, dtype: int64


In [151]:
#Number of venues in each venue category
# dhaka_venues_merged['Venue Category'].value_counts()

In [152]:
# clustered_venues = clusterd_venues.iloc[:,0:6]
# clustered_venues.reset_index(inplace=True)
# clustered_venues

In [153]:
# import seaborn as sns

In [154]:
# temp = dhaka_venues_merged['Venue Category'].value_counts()
# temp = pd.DataFrame(temp)
# temp.rename(columns={'Venue Category':'Count'})
# sns.distplot(temp)

## Results and Discussion <a name="results"></a>

So the cluster analysis results in 5 clusters of neighborhoods present in the city of Dhaka. To select the thanas that would be perfect for opening a coffee shop I checked all of the clusters and came to a conclusion that the possible best choice will be **cluster 1** and **cluster 3**. 
That is because we can say the places are more popular because of the abundance of the number of venues present in those areas. So in business perspective, those places will be perfect to open a coffee shop.

## Conclusion <a name="conclusion"></a>

Although the dataset consists of neighborhood data of every city in Bang and the foursquare API has been used to find out all the venues residing in those neighborhoods, but lack of population data, population density data in the neighborhoods certainly limit the capability to get a proper analysis of the business potential of each neighborhood. But, based on the current data, it can be said that, **Downsview** is a good choice to open a coffee shop in the city of Vaughan. 